# Simulation experiment 

Run entire simulation experiment multiple times to generate confidence interval

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Parameters
dataset_name = "Pseudomonas_analysis"
analysis_name = 'analysis_0'
NN_architecture = 'NN_2500_30'
file_prefix = 'Experiment_corrected'
num_simulated_samples = 6000
lst_num_experiments = [1, 2, 5, 10, 20,
                     50, 100, 500, 1000, 2000, 3000, 6000]
corrected = True
use_pca = True
num_PCs = 10

iterations = range(5) 
num_cores = 5

In [3]:
# Input files
base_dir = os.path.abspath(
  os.path.join(
      os.getcwd(), "../.."))    # base dir on repo


normalized_data_file = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "input",
    "train_set_normalized.pcl")

In [4]:
# Output files
similarity_corrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "analysis_0_similarity_corrected.pickle")

ci_corrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "analysis_0_ci_corrected.pickle")

In [5]:
# Run multiple simulations
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.simple_simulation_experiment_corrected)(i,
                                                          NN_architecture,
                                                          dataset_name,
                                                          analysis_name,
                                                          num_simulated_samples,
                                                          lst_num_experiments,
                                                          corrected,
                                                          use_pca,
                                                          num_PCs,
                                                          file_prefix,
                                                          normalized_data_file) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 173.1min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 173.2min remaining: 259.8min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 173.6min remaining: 115.7min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 173.6min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 173.6min finished


In [6]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of experiments,,,,,
1,0.999994,0.999994,0.999994,0.999994,0.999994
2,0.999826,0.999876,0.999915,0.999922,0.999919
5,0.999731,0.999723,0.999491,0.999629,0.999764
10,0.999213,0.999183,0.999257,0.999248,0.999200
20,0.998535,0.998463,0.998472,0.998244,0.998204
50,0.996192,0.995795,0.995878,0.995766,0.995318
100,0.991518,0.991695,0.991157,0.991352,0.991329
500,0.957571,0.955829,0.958456,0.957020,0.957445
1000,0.911666,0.913189,0.910866,0.913038,0.911606


In [7]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of experiments,
1,0.999994
2,0.999891
5,0.999668
10,0.999220
20,0.998384
50,0.995790
100,0.991410
500,0.957264
1000,0.912073


In [8]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of experiments,
1,6.819684e-08
2,1.300237e-05
5,3.495182e-05
10,1.002780e-05
20,4.706895e-05
50,9.909318e-05
100,6.457542e-05
500,3.027294e-04
1000,3.168509e-04


In [9]:
# Get confidence interval for each row (number of experiments)
# z-score for 95% confidence interval
err = std_scores*1.96

In [10]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of experiments,,
1,0.999994,0.999994
2,0.999866,0.999917
5,0.999599,0.999736
10,0.999201,0.999240
20,0.998291,0.998476
50,0.995596,0.995984
100,0.991284,0.991537
500,0.956671,0.957858
1000,0.911452,0.912694


In [11]:
mean_scores

,score
number of experiments,
1,0.999994
2,0.999891
5,0.999668
10,0.999220
20,0.998384
50,0.995790
100,0.991410
500,0.957264
1000,0.912073


In [12]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_corrected_file)
ci.to_pickle(ci_corrected_file)